# **SNOWFLAKE CORTEX SEARCH FINANCIAL SERVICES DEMO**
## Authors: John Heisler, Garrett Frere
In this demo, using Snowflake Cortex (https://www.snowflake.com/en/data-cloud/cortex/), we will build an RAG-powered chat interface using Cortex Search and Streamlit in Snowflake.

### RAG
We'll learn how to extract raw text from a PDF, chunk the raw text, perform prompt engineering, pass custom prompts and data to a large language model, and use Cortex Search to both handle our embeddings and retreval all without leaving Snowflake.

Specifically, we will be taking on the role of an AI Engineer who is working closely with a portfolio team at an asset manager. The portfolio team would like to deepend their comprehension of Federal Open Market Committee (FOMC) statements and meeting minutes. The FOMC determines the direction of monetary policy by directing open market operations. Ultimately the portfolio team would like an interface to ask and answer specific questions of a document wihtout searching throughout the text.

In this section we'll: 
1. Create a new table function to extract and chunk the raw text from the statements and meeting minutes (which are in pdfs).
2. Chunk and load raw text from the statements and minutes into our table.
3. Create a Cortex Search Service to handle the vectorization and search functionlity.
4. Create a Chat interface with Streamlit in Snowflake.

This is meant to be a companion to the FSI_Cortex_AI_Pipeline.ipynb also in this repository. 

# RAG
![RAG](https://publish-p57963-e462109.adobeaemcloud.com/adobe/dynamicmedia/deliver/dm-aid--7e5d3595-a32c-44de-86ca-cfa2883d475e/rag1.png?quality=85&width=1920&preferwebp=true 'CORTEX SEARCH')


# RAG with Cortex Search
![Cortex Search](https://quickstarts.snowflake.com/guide/ask_questions_to_your_own_documents_with_snowflake_cortex_search/img/1d96fe59a89a3ac5.png)



# 🛑 BEFORE YOU START 🛑

**Run the 1_SQL_SETUP_FOMC.sql script**


# RAG - Step 1 
## Chunking Function
Earlier, we created a function that pulled all of the text out of a PDF. Now, we'll do somethign very similar but we're going to break the text up into chunks for fuel our RAG interface. 

In [ ]:
--create a function to chunk the pdfs
CREATE OR REPLACE FUNCTION PDF_TEXT_CHUNKER(file_url string)
RETURNS TABLE (chunk varchar)
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
HANDLER = 'pdf_text_chunker'
PACKAGES = ('snowflake-snowpark-python','PyPDF2', 'langchain')
AS
$$
from snowflake.snowpark.types import StringType, StructField, StructType
from langchain.text_splitter import RecursiveCharacterTextSplitter
from snowflake.snowpark.files import SnowflakeFile
import PyPDF2, io
import logging
import pandas as pd

class pdf_text_chunker:

    def read_pdf(self, file_url: str) -> str:
    
        logger = logging.getLogger("udf_logger")
        logger.info(f"Opening file {file_url}")
    
        with SnowflakeFile.open(file_url, 'rb') as f:
            buffer = io.BytesIO(f.readall())
            
        reader = PyPDF2.PdfReader(buffer)   
        text = ""
        for page in reader.pages:
            try:
                text += page.extract_text().replace('\n', ' ').replace('\0', ' ')
            except:
                text = "Unable to Extract"
                logger.warn(f"Unable to extract from file {file_url}, page {page}")
        
        return text

    def process(self,file_url: str):

        text = self.read_pdf(file_url)
        
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size = 500, #Adjust this as you see fit
            chunk_overlap  = 50, #This let's text have some form of overlap. Useful for keeping chunks contextual
            length_function = len
        )
    
        chunks = text_splitter.split_text(text)
        df = pd.DataFrame(chunks, columns=['chunks'])
        
        yield from df.itertuples(index=False, name=None)
$$;

# RAG - STEP 2 
## Build the Chunk Table
Using our newly create chunking table function, being in and chunk all of the documents.

In [ ]:
TRUNCATE TABLE PDF_CHUNKS;
INSERT INTO PDF_CHUNKS (ID, FULL_TEXT_FK, RELATIVE_PATH, FILE_DATE, CHUNK)
WITH chunk_cte AS (
SELECT 
    FED_PDF_CHUNK_SEQUENCE.NEXTVAL as ID,
    relative_path,
    REPLACE(func.chunk,'''' ,'') as chunk,
FROM 
    directory(@FED_PDF),
    TABLE(pdf_text_chunker(build_scoped_file_url(@FED_PDF, relative_path))) as func
)

SELECT
    cte.ID,
    pft.ID,
    cte.relative_path, 
    pft.file_date,
    cte.chunk
FROM
    chunk_cte cte
LEFT JOIN 
    PDF_FULL_TEXT pft
    on  cte.relative_path = pft.RELATIVE_PATH;

# RAG - Step 3
## Create a Cortex Search Service

Cortex Search enables low-latency, high-quality “fuzzy” search over your Snowflake data. Cortex Search powers a broad array of search experiences for Snowflake users including Retrieval Augmented Generation (RAG) applications leveraging Large Language Models (LLMs).

We'll use this service later to power our RAG application. 

In [ ]:
--create a cortex Search Service 
CREATE OR REPLACE CORTEX SEARCH SERVICE SRCH_FED
ON CHUNK
ATTRIBUTES ID, FILE_DATE
WAREHOUSE = GEN_AI_FSI_WH
TARGET_LAG = '1 day'
AS (
    SELECT 
        ID,
        FILE_DATE::string as FILE_DATE,
        CHUNK AS CHUNK  
FROM PDF_CHUNKS);

# RAG - STEP 4
## FOMC Chat Interface
* We're leveraging our Cortex Search service enabling users to ask targeted questions of the documents in their stage.
* a robust chat interface could be built to handle this, for the demo, we have a bare bones interaction.

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.core import Root
import streamlit as st
import json5 as json
import pandas as pd
#import snowflake.snowpark.modin.plugin

#get our session
session = get_active_session()

# access search service through Python API
root = Root(session)
search_service = (root
                  .databases["GEN_AI_FSI"]
                  .schemas["FOMC"]
                  .cortex_search_services["SRCH_FED"]    
)

#create a function to generate response
def complete_cs(model_name, prompt):
    cmd = f"""SELECT SNOWFLAKE.CORTEX.TRY_COMPLETE('{model_name}','{prompt}') as response"""
    df_response = session.sql(cmd).collect()
    response = df_response[0].RESPONSE
    return response


#get FOMC files
database = 'GEN_AI_FSI'
schema = 'FOMC'

#USER INPUT: select time frame
query_document_dates = f"""SELECT DISTINCT FILE_DATE FROM {database}.{schema}.PDF_CHUNKS order by file_date desc;"""
df_document_dates = session.sql(query_document_dates).to_pandas()

#USER INPUT: select model
query_models = f"""SELECT MODEL FROM {database}.{schema}.MODELS"""
df_models = session.sql(query_models).to_pandas()

#USER INPUT: display
col1, col2 = st.columns(2)
with col1:
    user_input_date = st.selectbox("Select Document Date", df_document_dates, key="CS_date_select_box")
with col2: 
    user_input_model = st.selectbox("Select Model", df_models, key="CS_model_select_box")

#Generate a response
user_input_question = st.text_input("Ask me a question")

ask= st.button("Ask", key = "button_ask")
if ask: 
    #get the cunks that are relevant to the question
    response = search_service.search(
        user_input_question,
        columns=["ID", "FILE_DATE", "CHUNK"],
        filter={"@eq": {"FILE_DATE": f"""{user_input_date}"""} },
        limit=5,
    ).to_json()

    #st.json(response)
    # Parse the JSON5 string
    context_chunks = json.loads(response)
    
    #transform the data into a single string
    context_full = ""
    for chunk in context_chunks['results']:
        context_full += chunk['CHUNK'] + " "

    #build our prompt
    cs_prompt = f'''
            Role: You are an expert Senior Economist deeply knowledgeable on Federal Reserve documents and guidance including FOMC or Federal Open Market Committee 
            meeting minutes and communications. You are an expert in interpreting and answering investment-related questions based on meeting minutes and communications 
            which you are provided as context with each question.
            
            Data: You are provided with relevant text of a Federal Reserve Guidance or FOMC meeting notes relenavt to the question asked. 
            These meeting notes are generally released before the Federal Reserve takes action on economic policy.
            
            Task: Follow these instructions,
            1) Answer the question based on the context. 
            2) Be terse and do not consider information outside what you have been provided in the question and context.
            
            Output: produce thourough, valid, gramtically correct, and concise response in a professional tone. Please do not preface your response. also provide the document and location you used to answer the question.
            Context: {context_full}
            Question: Based on documents released on this date {user_input_date}, {user_input_question} 
            '''

    data = complete_cs(user_input_model, cs_prompt)
    
    with st.chat_message("model", avatar ="assistant"):
        st.write(data)